### *This notebook consolidates all other Pokemon data notebooks and serves as the final end-user tool to query a variety of Pokemon data. To use the tool, select 'Run', then 'Run All Cells' in the menu bar above, then scroll to the bottom and select whichever info wished to be displayed. 

#### Run these commands in the terminal to ensure ipywidgets is installed

In [3]:
#pip install ipywidgets
#jupyter nbextension enable --py widgetsnbextension
#!pip show ipywidgets  # Validate ipywidgets (as needed)

### Run repository notebooks to create widgets from

In [6]:
%run '02 - National Pokedex.ipynb'

In [7]:
%run '03 - Movesets and Natures.ipynb'

In [8]:
%run '04 - Items and TMs.ipynb'

In [26]:
%run '05 - Types.ipynb'

In [11]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image, display
pd.options.mode.chained_assignment = None  # default='warn'
# Maximize display of all dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### Press shift+enter to load widgets

In [55]:
# Key matching game to regional pokedex columns in national pokedex dataframe for subsequent filtering 
regional_pokedex_dict={
    'FireRed/LeafGreen': 'Kanto Pokedex',
    'Ruby/Sapphire/Emerald': 'Hoenn Pokedex',
    'Diamond/Pearl/Platinum': 'Sinnoh Pokedex',
    'HeartGold/SoulSilver': 'Johto Pokedex',
    'Black/White': 'Unova Pokedex (Black/White)',
    'Black2/White2': 'Unova Pokedex (Black2/White2)'
}
# Key matching game to corresponding pokemon moveset dictionaries
pokemon_movesets_dict={
    'FireRed/LeafGreen': gen3_movesets,
    'Ruby/Sapphire/Emerald': gen3_movesets,
    'Diamond/Pearl/Platinum': gen4_movesets,
    'HeartGold/SoulSilver': gen4_movesets,
    'Black/White': gen5_movesets,
    'Black2/White2': gen5_movesets
}
# Key matching game to corresponding move dictionaries
move_type_dict={
    'FireRed/LeafGreen': gen3_moves,
    'Ruby/Sapphire/Emerald': gen3_moves,
    'Diamond/Pearl/Platinum': gen4_moves,
    'HeartGold/SoulSilver': gen4_moves,
    'Black/White': gen5_moves,
    'Black2/White2': gen5_moves
}
# Key matching game to corresponding move dictionaries
items_dict={
    'Held Items': held_items,
    'General Items': general_items,
    'Berries': berries,
    'Battle Items': battle_items,
    'Medicine': medicine,
    'Pokeballs': pokeballs,
    # TMs unique to each game
    'FireRed/LeafGreen': FRLG_df,
    'Ruby/Sapphire/Emerald': RSE_df,
    'Diamond/Pearl/Platinum': DPP_df,
    'HeartGold/SoulSilver': HGSS_df,
    'Black/White': BW_df,
    'Black2/White2': BW2_df
}
# Define an image scraping function
def scrape_image(pokemon):
    url = f'https://img.pokemondb.net/sprites/home/normal/{pokemon}.png'
    display(HTML(f'<div style="text-align: center;"><img src="{url}" alt="{pokemon}"></div>'))
    
# Select game first
game_dropdown=widgets.Dropdown(
    options=['--Select from dropdown--','FireRed/LeafGreen','Ruby/Sapphire/Emerald','Diamond/Pearl/Platinum',
             'HeartGold/SoulSilver','Black/White','Black2/White2'],
    description='Which game are you playing?',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
# Initial dropdown options 
action_dropdown=widgets.Dropdown(
    options=['--Select from dropdown--','View Regional Pokedex','View Pokemon Stats','View National Pokedex',
             'Select Pokemon Moveset','View Pokemon Natures','View Moves by Type','View Type Matchup','View Pokemon Matchup','View Items'],
    description='What do you want to view?',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
# Post action dropdown selections
# Individual Pokemon stats
pokemon_stats_input=widgets.Text(
    description='Enter Pokemon whose stats you want to see:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'},  # Ensures full description is visible
)    
# Individual Pokemon moveset
pokemon_moveset_input=widgets.Text(
    description='Enter Pokemon whose moveset you want to see:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
# Moves by type
type_dropdown=widgets.Dropdown(
    options=['--Select from dropdown--','Bug', 'Dark', 'Dragon', 'Electric', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 
             'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Water'],
    description='Select Type:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
) 
# Items
item_dropdown=widgets.Dropdown(
    options=['--Select from dropdown--','Held Items','General Items','Berries','Battle Items','Medicine','Pokeballs','TMs'],
    description='Select Item Category:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
) 
# Type input
type_input=widgets.Text(
    description='Enter type:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
# Type matchup
type_matchup=widgets.Text(
    description='Enter type to see matchup:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
# Pokemon/Opponent matchup
pokemon_matchup=widgets.Text(
    description='Enter Pokemon:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
opponent_matchup=widgets.Text(
    description='Enter opponent Pokemon:',
    layout=widgets.Layout(width='max-content'),  # Expands the dropdown width
    style={'description_width': 'max-content'}   # Ensures full description is visible
)
output=widgets.Output()

# Display upon action dropdown selection
def update_action(change):
    action=action_dropdown.value  
    with output:
        output.clear_output()
        if action=='View Pokemon Stats':
            display(pokemon_stats_input)
        elif action=='Select Pokemon Moveset':
            display(pokemon_moveset_input)
        elif action=='View Moves by Type':
            display(type_dropdown)
        elif action=='View Type Matchup':
            display(type_input)
            display(type_matchup)
        elif action=='View Pokemon Matchup':
            display(pokemon_matchup)
            display(opponent_matchup)
        elif action=='View Regional Pokedex' or action=='View National Pokedex' or action=='View Pokemon Natures':
            display_results(None)  # Trigger display_results for these actions
        elif action=='View Items':
            display(item_dropdown)
        else:
            output.clear_output()
            
def display_results(change):
    with output:
        output.clear_output()
        # Define user input variables to create widgets from
        game=game_dropdown.value  # Game dropdown selection
        action=action_dropdown.value  # Action dropdown selection
        # Post action dropdown selections 
        pokemon_stats=pokemon_stats_input.value.strip()
        pokemon_moveset=pokemon_moveset_input.value.strip()
        type_input_value=type_input.value.strip()
        type_matchup_value=type_matchup.value.strip()
        pokemon_matchup_input=pokemon_matchup.value.strip()
        opponent_matchup_input=opponent_matchup.value.strip()
        item_input=item_dropdown.value
        move_type=type_dropdown.value.lower()
        regional_pokedex=regional_pokedex_dict[game]  # Regional Pokedex for whichever game is selected
                
        if action=='View Regional Pokedex':
            filtered_pokedex=pokedex_df[pokedex_df[regional_pokedex]==True] # Filter national pokedex to include only selected region pokedex
            filtered_pokedex=filtered_pokedex.iloc[:, :11]
            display(HTML(f"<div style='text-align: left; font-weight: bold; font-size: 24px;'>{game} Regional Pokedex</div>"))
            display(filtered_pokedex)
        elif action=='View Type Matchup':
            # Initialize pokemon and opponent dictionaries from user input
            type_input_dict = globals()[f'{type_input_value.lower()}_type']  
            type_matchup_dict=globals()[f'{type_matchup_value.lower()}_type']
            if type_matchup_value in type_input_dict['Super Effective']:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} is super effective against {type_matchup_value}.</div>"))
            elif type_matchup_value in type_input_dict['Not Very Effective']:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} is not very effective against {type_matchup_value}.</div>"))
            elif type_matchup_value in type_input_dict['No Effect']:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} has no effect against {type_matchup_value}.</div>"))
            else:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} has regular effect against {type_matchup_value}.</div>"))
            if type_matchup_value in type_input_dict['Weak To']:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} is weak against {type_matchup_value}.</div>"))
            elif type_matchup_value in type_input_dict['Resists']:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} resists {type_matchup_value}.</div>"))
            elif type_matchup_value in type_input_dict['Immune To']:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} is not affected by {type_matchup_value}.</div>"))
            else:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 16px;'>{type_input_value.capitalize()} is affected regularly by {type_matchup_value}.</div>"))
            print('')
            display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{type_input_value.capitalize()} Type Attributes</div>"))
            type_input_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in type_input_dict.items()])).fillna('')
            display(type_input_df)
        elif action=='View Pokemon Matchup':
            pokemon_matchup_url=f'https://img.pokemondb.net/sprites/home/normal/{pokemon_matchup_input}.png'
            opponent_matchup_url=f'https://img.pokemondb.net/sprites/home/normal/{opponent_matchup_input}.png'
            filtered_pokedex=pokedex_df[pokedex_df[regional_pokedex]==True]
            # Calculate offensive and defensive scores and display matchup from matchup algorithm
            # Include conditions to account for errored user inputs (e.g. user enters pokemon not in selected game, user enters pokemon that doesn't exist)
            if pokemon_matchup_input.upper() in filtered_pokedex['Pokémon'].values and opponent_matchup_input.upper() in filtered_pokedex['Pokémon'].values:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{pokemon_matchup_input.capitalize()}</div>"))
                display(HTML(f'<div style="text-align: left;"><img src="{pokemon_matchup_url}" alt="{pokemon_matchup_input}"></div>'))
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{opponent_matchup_input.capitalize()}</div>"))
                display(HTML(f'<div style="text-align: left;"><img src="{opponent_matchup_url}" alt="{opponent_matchup_input}"></div>'))
                matchup(pokemon_matchup_input, opponent_matchup_input)
            elif pokemon_matchup_input.upper() in filtered_pokedex['Pokémon'].values and opponent_matchup_input.upper() not in filtered_pokedex['Pokémon'].values and pokemon_matchup_input.lower() in pokemon_movesets_dict[game] and opponent_matchup_input.lower() in pokemon_movesets_dict[game]:
                print(f'{opponent_matchup_input.capitalize()} does not exist in the {game} regional pokedex. However, it does exist in the {game} national pokedex:')
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{pokemon_matchup_input.capitalize()}</div>"))
                display(HTML(f'<div style="text-align: left;"><img src="{pokemon_matchup_url}" alt="{pokemon_matchup_input}"></div>'))
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{opponent_matchup_input.capitalize()}</div>"))
                display(HTML(f'<div style="text-align: left;"><img src="{opponent_matchup_url}" alt="{opponent_matchup_input}"></div>'))
                matchup(pokemon_matchup_input, opponent_matchup_input)
            elif pokemon_matchup_input.upper() not in filtered_pokedex['Pokémon'].values and opponent_matchup_input.upper() in filtered_pokedex['Pokémon'].values and pokemon_matchup_input.lower() in pokemon_movesets_dict[game] and opponent_matchup_input.lower() in pokemon_movesets_dict[game]:
                print(f'{pokemon_matchup_input.capitalize()} does not exist in the {game} regional pokedex. However, it does exist in the {game} national pokedex:')
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{pokemon_matchup_input.capitalize()}</div>"))
                display(HTML(f'<div style="text-align: left;"><img src="{pokemon_matchup_url}" alt="{pokemon_matchup_input}"></div>'))
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{opponent_matchup_input.capitalize()}</div>"))
                display(HTML(f'<div style="text-align: left;"><img src="{opponent_matchup_url}" alt="{opponent_matchup_input}"></div>'))
                matchup(pokemon_matchup_input, opponent_matchup_input)
            elif pokemon_matchup_input.upper() not in filtered_pokedex['Pokémon'].values and pokemon_matchup_input.upper() in pokedex_df['Pokémon'].values:
                print(f'{pokemon_matchup_input.capitalize()} does not exist yet in this game.')
            elif opponent_matchup_input.upper() not in filtered_pokedex['Pokémon'].values and opponent_matchup_input.upper() in pokedex_df['Pokémon'].values:
                print(f'{opponent_matchup_input.capitalize()} does not exist yet in this game.')
            elif pokemon_matchup_input.upper() not in pokedex_df['Pokémon'].values:
                print(f'{pokemon_matchup_input.capitalize()} does not exist in any of these games.')
            elif opponent_matchup_input.upper() not in pokedex_df['Pokémon'].values:
                print(f'{opponent_matchup_input.capitalize()} does not exist in any of these games.')
        elif action=='View Pokemon Stats':
            filtered_pokedex=pokedex_df[pokedex_df[regional_pokedex]==True]  # Filter national pokedex to include only selected region pokedex
            # Make sure user input exists in game regional pokedex
            if pokemon_stats.upper() in filtered_pokedex['Pokémon'].values:
                row=filtered_pokedex[filtered_pokedex['Pokémon'] == pokemon_stats.upper()]
                row['Generation']=row['Generation'].astype('int')
                scrape_image(pokemon_stats)
            # Create and display base stat visuals for selected pokemon
                # Specify Pokemon stats to display
                columns_to_plot=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
                values = row[columns_to_plot].iloc[0].astype('int')  # Extract the values from the row                
                # Specify color shading by stat value
                norm=mcolors.Normalize(vmin=min(values), vmax=max(values))
                cmap=plt.colormaps['Greens']
                colors=[mcolors.rgb2hex(cmap(norm(v))[:3]) for v in values]                
                # Create subplot layout
                fig=make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'domain'}]])                
                # Add Bar Chart
                fig.add_trace(go.Bar(
                        x=columns_to_plot,
                        y=values.tolist(),
                        marker=dict(color=colors)),
                    row=1, col=1)                
                # Add Pie Chart
                fig.add_trace(go.Pie(
                        labels=columns_to_plot,
                        values=values.tolist(),
                        marker=dict(colors=colors)),
                    row=1, col=2)              
                # Update layout
                fig.update_layout(
                    title_text=f'{pokemon_stats.capitalize()} Base Stat Distribution',
                    title_font=dict(size=24, color='black'),
                    title_x = 0.5,
                    showlegend=False,
                    height=500,
                    width=1100)
                fig.show()
                display(row)  # Display pokedex entry for selected pokemon  
            else:
                # User input exceptions 
                if pokemon_stats.lower() in pokemon_movesets_dict[game]:
                    print(f'{pokemon_stats.capitalize()} does not exist in the {game} regional pokedex. However, here is its stats in the {game} national pokedex:')
                    row=pokedex_df[pokedex_df['Pokémon']==pokemon_stats.upper()]
                    scrape_image(pokemon_stats)
                # Create and display base stat visuals for selected pokemon
                    # Specify Pokemon stats to display
                    columns_to_plot=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
                    values = row[columns_to_plot].iloc[0].astype('int')  # Extract the values from the row                
                    # Specify color shading by stat value
                    norm=mcolors.Normalize(vmin=min(values), vmax=max(values))
                    cmap=plt.colormaps['Greens']
                    colors=[mcolors.rgb2hex(cmap(norm(v))[:3]) for v in values]                
                    # Create subplot layout
                    fig=make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'domain'}]])                
                    # Add Bar Chart
                    fig.add_trace(go.Bar(
                            x=columns_to_plot,
                            y=values.tolist(),
                            marker=dict(color=colors)),
                        row=1, col=1)                
                    # Add Pie Chart
                    fig.add_trace(go.Pie(
                            labels=columns_to_plot,
                            values=values.tolist(),
                            marker=dict(colors=colors)),
                        row=1, col=2)              
                    # Update layout
                    fig.update_layout(
                        title_text=f'{pokemon_stats.capitalize()} Base Stat Distribution',
                        title_font=dict(size=24, color='black'),
                        title_x = 0.5,
                        showlegend=False,
                        height=500,
                        width=1100)
                    fig.show()
                    display(row)
                elif pokemon_stats.upper() not in pokedex_df['Pokémon'].values:
                    print(f'{pokemon_stats.capitalize()} does not exist in any of these games')
                else:
                    print(f'{pokemon_stats.capitalize()} does not exist yet in this game.')
        elif action=='Select Pokemon Moveset':
            filtered_pokedex=pokedex_df[pokedex_df[regional_pokedex]==True]  # Filter national pokedex to include only selected region pokedex
            # Make sure user input exists in game regional pokedex          
            if pokemon_moveset.upper() in filtered_pokedex['Pokémon'].values:
                url = f'https://img.pokemondb.net/sprites/home/normal/{pokemon_moveset}.png'
                display(HTML(f'<div style="text-align: left;"><img src="{url}" alt="{pokemon_moveset}"></div>'))
                display(HTML(f"<div style='text-align: left; font-weight: bold; font-size: 24px;'>{pokemon_moveset.capitalize()} Moveset</div>"))
                display(pokemon_movesets_dict[game][pokemon_moveset])  
            else:
                # User input exceptions 
                if pokemon_moveset in pokemon_movesets_dict[game]:
                    print(f'{pokemon_moveset.capitalize()} does not exist in the {game} regional pokedex. However, it does exist in the national pokedex.')
                    url = f'https://img.pokemondb.net/sprites/home/normal/{pokemon_moveset}.png'
                    display(HTML(f'<div style="text-align: left;"><img src="{url}" alt="{pokemon_moveset}"></div>'))
                    display(HTML(f"<div style='text-align: left; font-weight: bold; font-size: 24px;'>{pokemon_moveset.capitalize()} Moveset</div>"))
                    display(pokemon_movesets_dict[game][pokemon_moveset])
                elif pokemon_moveset.upper() not in pokedex_df['Pokémon'].values:
                    print(f'{pokemon_moveset.capitalize()} does not exist in any of these games.')
                else:
                    print(f'{pokemon_moveset.capitalize()} does not exist yet in this game.')
        elif action=='View National Pokedex':
            if game=='FireRed/LeafGreen':
                filtered_pokedex=pokedex_df[(pokedex_df['Generation']=='1')|(pokedex_df['Generation']=='2')]
            elif game=='Ruby/Sapphire/Emerald':
                filtered_pokedex=pokedex_df[(pokedex_df['Generation']=='1')|(pokedex_df['Generation']=='2')|(pokedex_df['Generation']=='3')]
            elif game=='Diamond/Pearl/Platinum':
                filtered_pokedex=pokedex_df[(pokedex_df['Generation']=='1')|(pokedex_df['Generation']=='2')|(pokedex_df['Generation']=='3')|
                                            (pokedex_df['Generation']=='4')] 
            elif game=='HeartGold/SoulSilver':
                filtered_pokedex=pokedex_df[(pokedex_df['Generation']=='1')|(pokedex_df['Generation']=='2')|(pokedex_df['Generation']=='3')|
                                            (pokedex_df['Generation']=='4')]  
            else:
                filtered_pokedex=pokedex_df[(pokedex_df['Generation']=='1')|(pokedex_df['Generation']=='2')|(pokedex_df['Generation']=='3')|
                                            (pokedex_df['Generation']=='4')|(pokedex_df['Generation']=='5')] 
            display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{game} National Pokedex</div>"))
            display(filtered_pokedex)            
        elif action=='View Pokemon Natures':
            display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>Natures</div>"))
            display(natures)
        elif action=='View Moves by Type':
            display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{move_type.capitalize()} Type Moves</div>"))
            display(move_type_dict[game][move_type])
        elif action=='View Items':
            if item_input=='TMs':
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{game} {item_input}</div>"))
                display(items_dict[game])
            else:
                display(HTML(f"<div style='text-align: Left; font-weight: bold; font-size: 24px;'>{item_input}</div>"))
                display(items_dict[item_input])
            
action_dropdown.observe(update_action, names='value')
pokemon_stats_input.on_submit(display_results)
pokemon_moveset_input.on_submit(display_results)
type_input.on_submit(display_results)
type_matchup.on_submit(display_results)
pokemon_matchup.on_submit(display_results)
opponent_matchup.on_submit(display_results)
type_dropdown.observe(display_results, names='value')
item_dropdown.observe(display_results, names='value')
display(game_dropdown, action_dropdown, output)

Dropdown(description='Which game are you playing?', layout=Layout(width='max-content'), options=('--Select fro…

Dropdown(description='What do you want to view?', layout=Layout(width='max-content'), options=('--Select from …

Output()